# 데이트 코칭 기능 구현하기

## 도움이 필요한 대화 하나 만들기

In [1]:
from typing import List
import random
import json

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

from tqdm.auto import tqdm

/Users/hyeonjinho/.pyenv/versions/3.11.6/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
model = ChatOpenAI(model='gpt-4-turbo-preview', temperature=0.)

In [3]:
class Turn(BaseModel):
    name: str = Field(description="이름")
    gender: str = Field(description="성별: '남자' 또는 '여자'")
    content: str = Field(description="발화")

In [4]:
class Conversation(BaseModel):
    turn_list: List[Turn] = Field(description="남녀의 대화, turn_list의 최대 길이는 6.")

In [5]:
parser = JsonOutputParser(pydantic_object=Conversation)
format_instructions = parser.get_format_instructions()
format_instructions

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"turn_list": {"title": "Turn List", "description": "남녀의 대화, turn_list의 최대 길이는 6.", "type": "array", "items": {"$ref": "#/definitions/Turn"}}}, "required": ["turn_list"], "definitions": {"Turn": {"title": "Turn", "type": "object", "properties": {"name": {"title": "Name", "description": "이름", "type": "string"}, "gender": {"title": "Gender", "description": "성별: \'남자\' 또는 \'여자\'", "type": "string"}, "content": {"title": "Content", "description": "발화", "type": "string"}}, "required": ["name", "gender", "content"]}}}\n```'

In [6]:
human_prompt_template = HumanMessagePromptTemplate.from_template(
                            "한국인 남녀 소개팅 상황에서의 대화를 만들어줘.\n"
                            "남자가 소개팅 상황에서의 대화를 잘 못해서 도움 필요한 상황을 만들어줘.\n{format_instructions}")

prompt = ChatPromptTemplate.from_messages(
    [
        human_prompt_template
    ])
prompt = prompt.partial(format_instructions=format_instructions)

In [7]:
conv_gen_chain = prompt | model | parser

In [8]:
conv = conv_gen_chain.invoke({})
conv

{'turn_list': [{'name': '지훈',
   'gender': '남자',
   'content': '안녕하세요, 처음 뵙겠습니다. 지훈이라고 합니다.'},
  {'name': '수연', 'gender': '여자', 'content': '안녕하세요, 수연입니다. 만나서 반가워요.'},
  {'name': '지훈', 'gender': '남자', 'content': '네, 저도요... 그, 그러니까... 취미가 뭐예요?'},
  {'name': '수연', 'gender': '여자', 'content': '저는 책 읽는 걸 좋아해요. 지훈 씨는요?'},
  {'name': '지훈',
   'gender': '남자',
   'content': '아, 저도 책 읽기... 좋아합니다. 그, 그런데 최근에 본 책이 기억이 안 나네요.'},
  {'name': '수연',
   'gender': '여자',
   'content': '괜찮아요, 나중에 생각나면 말해주세요. 저희 다음에 책 이야기도 나눠보고 싶네요.'}]}

## 대화 코칭 기능 만들기

In [9]:
human_prompt_template = HumanMessagePromptTemplate.from_template(
                            "{conv}\n위 소개팅 상황에서 {name}에게 소개팅을 잘 진행 할 수 있도록 조언해줘")

prompt = ChatPromptTemplate.from_messages(
    [
        human_prompt_template,
    ])

In [10]:
coach_chain = prompt | model | StrOutputParser()

In [11]:
coach_msg = coach_chain.invoke({"conv": conv, "name": "수연"})

In [12]:
print(coach_msg)

수연과의 소개팅에서 지훈이 좋은 인상을 남기고 대화를 잘 이어나갈 수 있도록 몇 가지 조언을 드리겠습니다.

1. **공통 관심사 활용하기**: 지훈이 책 읽기를 좋아한다고 했으니, 이를 공통 관심사로 활용하는 것이 좋습니다. 최근에 읽은 책이 기억나지 않더라도, 과거에 인상 깊게 읽었던 책이나 좋아하는 장르에 대해 이야기를 나누어 보세요. 이를 통해 수연과 더 깊은 대화를 나눌 수 있을 것입니다.

2. **경청하는 태도 보여주기**: 대화 중에는 수연이 말하는 내용에 집중하고 관심을 보여주세요. 수연이 좋아하는 책이나 취미에 대해 더 궁금한 점을 질문하거나, 그녀의 의견에 공감을 표현하는 것이 좋습니다. 상대방이 관심을 받고 있다고 느끼게 하는 것은 대화 분위기를 긍정적으로 만듭니다.

3. **자신감 있는 태도**: 대화를 이어가면서 자신감 있는 태도를 보여주세요. 물론 겸손함도 중요하지만, 자신의 의견이나 생각을 명확하게 표현하는 것도 중요합니다. 너무 긴장하지 말고, 자연스럽게 자신의 생각을 나누세요.

4. **다음 만남 제안하기**: 소개팅이 잘 진행되고 있다고 느껴진다면, 다음 만남을 제안해 보세요. 예를 들어, "다음에는 같이 책방에 가서 서로에게 추천해줄 만한 책을 고르는 건 어떨까요?"와 같은 방식으로 관심사를 기반으로 한 만남을 제안할 수 있습니다.

5. **진실된 모습 보여주기**: 자신을 있는 그대로 보여주는 것이 중요합니다. 소개팅에서는 상대방에게 좋은 인상을 남기고 싶은 마음에 때로는 자신을 과장하게 될 수도 있지만, 장기적인 관계를 위해서는 진실된 모습을 보여주는 것이 좋습니다.

지훈이 이러한 조언들을 참고하여 수연과의 소개팅을 더욱 즐겁고 의미 있는 시간으로 만들 수 있기를 바랍니다.
